In [52]:
rm(list=ls())

# Librerías ====
if(!require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, showtext, srvyr)

In [53]:
##Crear folders de almacenamiento
dir.create("indicadores/trabajo/microdatos", showWarnings = F)

In [54]:
# Función para descargar y descomprimir un archivo con timeout
download_and_unzip <- function(url, destination="indicadores/trabajo/microdatos/") {
  tryCatch({
    # Descargar el archivo con timeout
    download.file(url, destfile = paste0(destination, "/", basename(url)), mode = "wb", timeout = 10000)

    # Descomprimir el archivo
    unzip(paste0(destination, "/", basename(url)), exdir = destination)

    # Eliminar el archivo comprimido después de descomprimirlo
    unlink(paste0(destination, "/", basename(url)))

    message(paste("Descarga y descompresión exitosas para", basename(url)))
  }, error = function(e) {
    warning(paste("Error al descargar o descomprimir", basename(url), ":", conditionMessage(e)))
  })
}

In [55]:
download_and_unzip("https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos/enoe_2024_trim4_csv.zip")

Descarga y descompresión exitosas para enoe_2024_trim4_csv.zip



In [56]:
sdemt<-read_csv("indicadores/trabajo/microdatos/ENOE_SDEMT424.csv", show_col_types = FALSE)%>%
#Crear grupos de edad
mutate(gpoedad=case_when(eda>=15 & eda<=29 ~ "jovenes",
TRUE ~ "otros"))

In [57]:
dm <- as_survey_design(
    .data = sdemt,
    ids = upm,
    strata = est_d_tri,
    weights = fac_tri,
    nest = TRUE 
) 

In [58]:
dm%>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) %>%
summarize(ocupados = survey_total(vartype="cv"))


ocupados,ocupados_cv
<dbl>,<dbl>
5010045,0.01575094


In [59]:
dm%>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) %>%
group_by(gpoedad) %>%
summarize(ocupados = survey_total(vartype="cv"),
pct= survey_prop( vartype="cv")) %>%
ungroup() %>%
mutate(pct = pct*100) 


gpoedad,ocupados,ocupados_cv,pct,pct_cv
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
jovenes,1057829,0.03751445,21.11416,0.031319503
otros,3952216,0.01645091,78.88584,0.008382811


In [60]:
#Tasa de informalidad laboral
dm%>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) %>%
mutate(condinf=
case_when(mh_col %in% c(1,3,5,7,9) ~ "condiciones de informalidad",
TRUE ~ "condiciones de formalidad")) %>% 
group_by(gpoedad,condinf) %>%
summarize(ocupados = survey_total(vartype="cv"),
pct= survey_prop( vartype="cv")) %>%
ungroup() %>%
mutate(pct = pct*100) %>%
janitor::adorn_totals()

gpoedad,condinf,ocupados,ocupados_cv,pct,pct_cv
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
jovenes,condiciones de formalidad,519838,0.05223346,49.14197,0.03636801
jovenes,condiciones de informalidad,537991,0.05141782,50.85803,0.03514087
otros,condiciones de formalidad,2211975,0.02424996,55.96797,0.01804527
otros,condiciones de informalidad,1740241,0.02841056,44.03203,0.02293687
Total,-,5010045,0.15631179,200.00000,0.11249102


In [ ]:
#Tasa de condiciones críticas de ocupación
dm%>%
filter(r_def ==00, (c_res == 1 | c_res == 3),
         eda >= 15  & eda <= 98) %>%
filter(ent == 9, clase2 == 1) %>%
mutate(concrit=case_when(tcco ==0 ~ "Normales",
TRUE ~ "Criticas")) %>%
group_by(gpoedad,concrit) %>%
summarize(ocupados = survey_total(vartype="cv"),
pct= survey_prop( vartype="cv")) %>%
ungroup() %>%
mutate(pct = pct*100) %>%
janitor::adorn_totals()

gpoedad,concrit,ocupados,ocupados_cv,pct,pct_cv
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
jovenes,Criticas,347611,0.06655130,32.86079,0.05347480
jovenes,Normales,710218,0.04486615,67.13921,0.02617285
otros,Criticas,1044719,0.03849435,26.43375,0.03425682
otros,Normales,2907497,0.02021429,73.56625,0.01230913
Total,-,5010045,0.17012609,200.00000,0.12621360


: 